<a href="https://colab.research.google.com/github/MarcoE25/Clases-Analisis-Datos/blob/main/Trabajo_Clase_10_11_inter_202509.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resumen de lo visto en **Maestro en Pandas**

In [1]:
# PONER EN LA PRIMER CELDA TODAS LAS LIBRERÍAS
import pandas as pd
from dateutil import parser
import datetime as dt
import random as r
import numpy as np

# Webscraping
import urllib.request, json
from urllib.request import urlopen
from zipfile import ZipFile
import re
import requests

################################################################################
############################ CARGA DE LA INFORMACION ###########################
################################################################################
df_bank = pd.read_csv("https://cursopypagina.github.io/CursoPy/Clase_1.csv")
df_bank_ej = pd.read_csv("https://cursopypagina.github.io/CursoPy/Clase_1_5.csv")

# cambiar el nombre de una columna
#                                         columna anterior     : nombre nuevo de la columna
df_bank_ej = df_bank_ej.rename(columns={"Fecha_Inicio_Contrato": "Fecha_Inicio"})
df_bank_ej = df_bank_ej[["Nombre_Empresa", "Ejecutivo", "Fecha_Inicio"]]

################################################################################
############################ VALORES DEL USD-EUR ###############################
################################################################################
def divisa_webs(clave):
  url = "https://www.piplatam.com/Home/filiales?country=MX"
  page = urlopen(url)
  htmlBytes = page.read()
  html = htmlBytes.decode("utf-8")
  strValorClave = html.find(clave)
  stringFiltro1 = html[strValorClave:]
  strValor = stringFiltro1.split("dblValue")[1].split("dblChange")[0]
  strValor = re.findall(r"\d+\.\d+", strValor)[0]
  return strValor

usdValor = round(float(divisa_webs("USD-MXN FIX (Banxico)")), 2)
eurValor = round(float(divisa_webs("EUR-MXN")), 2)

################################################################################
########################## LIMPIEZA Y PROCESAMIENTO ############################
################################################################################
def clean_1(df_func, tipo):
  # si quieres mantener intactos tus insumos originales (lo recomendable)
  # hacemos una copia del dichos dataframes
  df_func_trabajo = df_func.copy()
  df_func_trabajo['Nombre_Empresa'] = df_func_trabajo['Nombre_Empresa'].str.strip().str.title()
  # apply: aplica una función a cada fila de una columna
  df_func_trabajo["Fecha_Inicio"] = df_func_trabajo["Fecha_Inicio"].apply(lambda x: parser.parse(x))

  if tipo == "Ejecutivo":
    ############################################################################
    # Agregamos la parte donde nos quedamos solo con un ejecutivo de ventas

    # "Dispersion": ["last"]
    df_func_trabajo = df_func_trabajo.groupby("Nombre_Empresa").agg(
        {
            "Fecha_Inicio": ['max'],
            "Ejecutivo": ['last']
        }
    )
    df_func_trabajo = df_func_trabajo.reset_index()
    # df_func_trabajo.rename(columns={"col1": 'Nombre_Empresa', "col2": 'Fecha_Inicio', "col3": Ejecutivo'})
    df_func_trabajo.columns = ['Nombre_Empresa', 'Fecha_Inicio', 'Ejecutivo']
    # eliminamos una columna
    del df_func_trabajo['Fecha_Inicio']
    return df_func_trabajo
  elif tipo == "General":
    df_func_trabajo['Fecha_Fin'] = df_func_trabajo['Fecha_Fin'].fillna('1900-01-01')
    df_func_trabajo['Plazo'] = df_func_trabajo['Plazo'].fillna(0)
    # astype cambiamos el tipo de dato de una columna
    df_func_trabajo['Plazo'] = df_func_trabajo['Plazo'].astype(int)

    df_func_trabajo['Fecha_Fin'] = df_func_trabajo['Fecha_Fin'].apply(lambda x: parser.parse(x))
    #                                  0       1
    # "Hola-Mundo".split("-") ---> ["Hola", "Mundo"]
    # "Hola"
    # "OP-1001".split("-")    ---> ["OP", "1001"]
    # "1001"
    df_func_trabajo['Operacion'] = df_func_trabajo['Operacion'].str.split("-").apply(lambda x: x[1])

    fechaReporte = dt.datetime.today() - dt.timedelta(days = 1)
    #                             f ----> convierte objetos fecha a strings fecha
    fechaReporte = dt.datetime.strftime(fechaReporte, "%Y-%m-%d")
    # Creamos una columna nueva con el mismo valor
    df_func_trabajo['Fecha_Reporte'] = fechaReporte

    # Columnas condicionales
    zonas = ["Norte", "Sur", "Centro"]

    # El zen de Python (la biblia de Python)
    # Un código más de Python ---> Pythonic
    # bucles ---> No son naturalmente Pythonicos
    # listas de comprehension si son Pythonicas
    df_func_trabajo['Zonas'] = [r.choice(zonas) for i in range(len(df_func_trabajo))]

    def valorizador(divisa):
      if divisa == "USD":
        return usdValor
      elif divisa == "EUR":
        return eurValor
      else:
        return 1
    df_func_trabajo['Divisa_Valor'] = df_func_trabajo['Divisa'].apply(valorizador)
    # Vectores en R    ---> "Vectores"
    # Arrays en Python ---> "Vectores"
    df_func_trabajo['Monto_Credito'] = df_func_trabajo['Monto_Credito'] * df_func_trabajo['Divisa_Valor']
    del df_func_trabajo['Divisa_Valor']

    return df_func_trabajo
  else:
    print("⚠️Nombre inválido⚠️")

################################################################################
##################### APLICAMOS LA FUNCION DE LIMPIEZA #########################
################################################################################
df_bank_clean = clean_1(df_bank, "General")
df_bank_clean_ej = clean_1(df_bank_ej, "Ejecutivo")

# Realizamos el merge
df_bank_final = df_bank_clean.merge(df_bank_clean_ej, on='Nombre_Empresa', how='left')
df_bank_final.head()

,Nombre_Empresa,Monto_Credito,Divisa,Operacion,Fecha_Inicio,Fecha_Fin,Plazo,Fecha_Reporte,Zonas,Ejecutivo
0,Grupo Bimbo,4.117590e+06,MXN,100000,2019-08-27,2023-04-24,1336,2025-09-29,Norte,Luis Manuel Ana María Vera Domínguez
1,Walmart De México,1.285973e+08,USD,100001,2017-04-10,2022-02-18,1775,2025-09-29,Sur,Eloisa Salinas Aguilar
2,Liverpool,2.134391e+08,EUR,100002,2017-04-21,1900-01-01,0,2025-09-29,Norte,Dr. Virginia Pelayo
3,Grupo Lala,2.372735e+07,USD,100003,2023-11-24,1900-01-01,0,2025-09-29,Sur,Juana Antonia Lira Samaniego
4,Femsa,1.534085e+08,EUR,100004,2024-10-30,1900-01-01,0,2025-09-29,Centro,Gabino Mariano Laureano Puente


---

# Preguntas de análisis

* ¿Cuál es el monto total de crédito otorgado por divisa?
* ¿Cuáles son las 5 empresas que han recibido más crédito en total?
* ¿Cuál es el monto total de créditos vigentes (sin Fecha_Fin real) por zona?


In [2]:
df_bank_final.columns

Index(['Nombre_Empresa', 'Monto_Credito', 'Divisa', 'Operacion',
       'Fecha_Inicio', 'Fecha_Fin', 'Plazo', 'Fecha_Reporte', 'Zonas',
       'Ejecutivo'],
      dtype='object')

In [3]:
# ¿Cuál es el monto total de crédito otorgado por divisa?
df_gby_divisa_monto = df_bank_final[['Divisa', 'Monto_Credito']].groupby('Divisa').sum().reset_index()
df_gby_divisa_monto['Monto_Credito_Format'] = df_gby_divisa_monto['Monto_Credito'].apply(lambda x: f"{x:,}")
df_gby_divisa_monto

,Divisa,Monto_Credito,Monto_Credito_Format
0,EUR,3.695581e+11,"369,558,119,434.5232"
1,MXN,1.650154e+10,"16,501,540,339.68"
2,USD,3.138121e+11,"313,812,089,827.91504"


In [4]:
# .sort_values(nombreColumna): te ordena tu dataframe con base en la columna nombreColumna
# se ordena de manera ascendente
df_gby_divisa_monto.sort_values("Monto_Credito")

,Divisa,Monto_Credito,Monto_Credito_Format
1,MXN,1.650154e+10,"16,501,540,339.68"
2,USD,3.138121e+11,"313,812,089,827.91504"
0,EUR,3.695581e+11,"369,558,119,434.5232"


In [5]:
# se ordena de manera descendente
df_gby_divisa_monto.sort_values("Monto_Credito", ascending=False)

,Divisa,Monto_Credito,Monto_Credito_Format
0,EUR,3.695581e+11,"369,558,119,434.5232"
2,USD,3.138121e+11,"313,812,089,827.91504"
1,MXN,1.650154e+10,"16,501,540,339.68"


In [6]:
# ¿Cuáles son las 5 empresas que han recibido más crédito en total?
df_gby_empresa_monto = df_bank_final[['Nombre_Empresa', 'Monto_Credito']].groupby('Nombre_Empresa').sum().reset_index()
df_gby_empresa_monto['Monto_Credito_Format'] = df_gby_empresa_monto['Monto_Credito'].apply(lambda x: f"{x:,}")

# pandas --> sort_values <-----> Order by
df_gby_empresa_monto.sort_values("Monto_Credito", ascending=False).head(5)

,Nombre_Empresa,Monto_Credito,Monto_Credito_Format
5,Femsa,5.119268e+10,"51,192,677,636.2797"
3,Bbva México,4.916227e+10,"49,162,265,698.1959"
14,Walmart De México,4.870038e+10,"48,700,376,044.3146"
2,Banorte,4.832633e+10,"48,326,325,221.5737"
11,Pemex,4.684953e+10,"46,849,534,442.5479"


In [7]:
# # ¿Cuáles son las 5 empresas que han recibido menos crédito en total?
# ¿Cuáles son las 5 empresas que han recibido más crédito en total?
df_gby_empresa_monto = df_bank_final[['Nombre_Empresa', 'Monto_Credito']].groupby('Nombre_Empresa').sum().reset_index()
df_gby_empresa_monto['Monto_Credito_Format'] = df_gby_empresa_monto['Monto_Credito'].apply(lambda x: f"{x:,}")
df_gby_empresa_monto.sort_values("Monto_Credito")

,Nombre_Empresa,Monto_Credito,Monto_Credito_Format
0,Alsea,4.138519e+10,"41,385,185,823.2034"
12,Soriana,4.417202e+10,"44,172,023,440.1931"
6,Grupo Bimbo,4.469775e+10,"44,697,754,530.5929"
9,Grupo México,4.611247e+10,"46,112,468,678.492905"
13,Telmex,4.636733e+10,"46,367,333,218.965004"
1,América Móvil,4.637805e+10,"46,378,052,121.5736"
7,Grupo Carso,4.651391e+10,"46,513,905,026.028404"
10,Liverpool,4.656203e+10,"46,562,033,312.2993"
8,Grupo Lala,4.669941e+10,"46,699,411,984.5105"
4,Cemex,4.675240e+10,"46,752,402,423.3473"


In [8]:
# ¿Cuál es el monto total de créditos vigentes por zona?
# Traernos todos los creditos vigentes
df_bank_final[df_bank_final['Fecha_Fin'] == '1900-01-01']

,Nombre_Empresa,Monto_Credito,Divisa,Operacion,Fecha_Inicio,Fecha_Fin,Plazo,Fecha_Reporte,Zonas,Ejecutivo
2,Liverpool,2.134391e+08,EUR,100002,2017-04-21,1900-01-01,0,2025-09-29,Norte,Dr. Virginia Pelayo
3,Grupo Lala,2.372735e+07,USD,100003,2023-11-24,1900-01-01,0,2025-09-29,Sur,Juana Antonia Lira Samaniego
4,Femsa,1.534085e+08,EUR,100004,2024-10-30,1900-01-01,0,2025-09-29,Centro,Gabino Mariano Laureano Puente
5,Grupo Lala,7.674495e+06,MXN,100005,2024-12-09,1900-01-01,0,2025-09-29,Centro,Juana Antonia Lira Samaniego
6,Grupo Carso,7.554816e+07,EUR,100006,2020-05-30,1900-01-01,0,2025-09-29,Sur,Gloria Otero Sáenz
...,...,...,...,...,...,...,...,...,...,...
9987,Walmart De México,1.770912e+08,EUR,109987,2021-03-26,1900-01-01,0,2025-09-29,Sur,Eloisa Salinas Aguilar
9988,Grupo Bimbo,9.586326e+06,MXN,109988,2021-11-17,1900-01-01,0,2025-09-29,Centro,Luis Manuel Ana María Vera Domínguez
9989,Pemex,1.315673e+08,USD,109989,2023-06-25,1900-01-01,0,2025-09-29,Centro,Guillermo Cristian Hernandes
9992,Pemex,1.053526e+08,USD,109992,2025-08-21,1900-01-01,0,2025-09-29,Centro,Guillermo Cristian Hernandes


In [9]:
# omitir las advertencias que suele dar Python
import warnings as w
w.filterwarnings("ignore")

# Creditos vigentes
seleccionColumna = ['Zonas', 'Monto_Credito']
df_temp_cred_vigentes = df_bank_final.query("Fecha_Fin == '1900-01-01'")[seleccionColumna]
df_gby_zonas_cred_vigentes = df_temp_cred_vigentes.groupby('Zonas').sum().reset_index()
df_gby_zonas_cred_vigentes['Monto_Credito'] = df_gby_zonas_cred_vigentes['Monto_Credito'].apply(round)
df_gby_zonas_cred_vigentes['Monto_Credito_Format'] = df_gby_zonas_cred_vigentes['Monto_Credito'].apply(lambda x: f"{x:,}")
df_gby_zonas_cred_vigentes

,Zonas,Monto_Credito,Monto_Credito_Format
0,Centro,147010182114,"147,010,182,114"
1,Norte,144996763125,"144,996,763,125"
2,Sur,146360810588,"146,360,810,588"


In [10]:
def cualquierCosa():
  # esta importacion es exclusiva para la funcion
  import matplotlib.pyplot as plt

# importacion para tu entorno de trabajo
import matplotlib.pyplot as plt

In [11]:
# importacion en el entorno de trabajo (libreta de python)
from sklearn.linear_model import LinearRegression

def cualquierCosa2():
  # se pueden acceder a las funcionalidades de esa importacion

SyntaxError: incomplete input (ipython-input-82075133.py, line 5)

* ¿Cuál es el monto total de crédito por zona y por divisa?
* ¿Cuántas operaciones se han realizado por cada empresa?
* ¿Cuál es el promedio de plazo de los créditos por zona?


In [ ]:
# dataframe.groupby("columnaCategorica")
# dataframe.groupby(["columnaCategorica1", "columnaCategorica2", ...])

In [ ]:
# ¿Cuál es el monto total de crédito por zona y por divisa?
df_gby_zon_div = df_bank_final[["Monto_Credito", "Divisa", "Zonas"]].groupby(["Zonas",  "Divisa"]).sum()
df_gby_zon_div = df_gby_zon_div.reset_index()
df_gby_zon_div

In [ ]:
# ¿Cuántas operaciones se han realizado por cada empresa?
df_bank_final.query("Nombre_Empresa == 'Grupo Bimbo'").head()

In [ ]:
# Paso previo: las operaciones sean todas diferentes
# value_counts("nombreColumnaCategorica"): te cuenta cuantos elementos
# diferentes tienes en tu tabla por categoria
df_bank_final.value_counts("Zonas")

In [ ]:
# cuantas veces aparece cada categoria en el dataframe
df_bank_final.value_counts("Divisa")

In [ ]:
# el unique sobre una columna te arroja todos los valores diferentes
# elimina duplicados
# te arroja los valores unicos
# te arroja los valores diferentes
df_bank_final["Divisa"].unique()

In [ ]:
df_bank_final.value_counts("Operacion").reset_index()['count'].unique()

In [ ]:
df_bank_final.value_counts("Operacion").reset_index()['count'].sum()

In [ ]:
# filas tienes, columnas tienes
df_bank_final.shape

In [ ]:
# ¿El conteo que estas haciendo es con valores unicos?
df_gby_oper = df_bank_final[["Nombre_Empresa", "Operacion"]].groupby("Nombre_Empresa").count()

# nombreDataFrame.reset_index(inplace=True)
df_gby_oper = df_gby_oper.reset_index()
df_gby_oper

---

In [ ]:
[
    "Luis",
    "Fernado"
]

In [ ]:
# En el dataframe final, se requiere ver cuantos creditos tiene cada empresa
# Tarea para el lector, agregar una columna con el monto total de deuda de cada empresa
# Agregar una columna de fecha_primera que contenga la fecha de inicio del primer
# credito que saco cada empresa (hint: last())
#######################################################################################

# agg({
#    "nombreColumna": ['last']
# })
# dataframe.groupby(<<>>).sum()
# dataframe.groupby(<<>>).count()
# dataframe.groupby(<<>>).last()
# dataframe.groupby(<<>>).max()
# dataframe.groupby(<<>>).min()
# dataframe.groupby(<<>>).mean()
df_gby_operacion_temp = df_bank_final[["Nombre_Empresa",
                                       "Operacion"]].groupby("Nombre_Empresa").count().reset_index()
df_bank_final = df_bank_final.merge(df_gby_operacion_temp, on="Nombre_Empresa", how='left')

In [ ]:
df_bank_final = df_bank_final.rename(columns={'Operacion_x': 'Operacion'})
df_bank_final = df_bank_final.rename(columns={'Operacion_y': 'Conteo_Creditos'})
df_bank_final

---

In [ ]:
# ¿Cuál es el promedio de plazo de los créditos por zona?
df_bank_final[["Zonas", "Plazo"]].groupby("Zonas").mean()

# Fecha de entrega: Máximo el 6 de octubre antes de las 7:30 pm

Las que quedan pendientes:

* ¿Qué zona ha recibido más créditos en total (suma del monto)?
* ¿Cuántas operaciones se han realizado en cada divisa?
* ¿Qué empresas tienen créditos activos (Fecha_Fin = 1900-01-01)?
* ¿Qué porcentaje del total de crédito corresponde a cada divisa?
* Agregar una columna con el monto total de deuda de cada empresa
* Agregar una columna de fecha_primera que contenga la fecha de inicio del primer credito que saco cada empresa (hint: last())

In [28]:
df_zonas_mas_credito = df_bank_final[["Zonas","Monto_Credito"]].groupby("Zonas")["Monto_Credito"].sum().reset_index().sort_values("Monto_Credito", ascending=False)
df_zonas_mas_credito["Monto_Credito"]=df_zonas_mas_credito["Monto_Credito"].apply(lambda x: f"{x:,}")
df_zonas_mas_credito

,Zonas,Monto_Credito
0,Centro,"235,976,207,041.11942"
2,Sur,"234,308,153,852.3181"
1,Norte,"229,587,388,708.6807"


In [33]:
df_operaciones_por_divisa=df_bank_final[["Operacion","Divisa"]].groupby("Divisa")["Operacion"].count().reset_index()
df_operaciones_por_divisa

,Divisa,Operacion
0,EUR,3392
1,MXN,3254
2,USD,3354


In [34]:
df_creditos_activos=df_bank_final[]

,Nombre_Empresa,Monto_Credito,Divisa,Operacion,Fecha_Inicio,Fecha_Fin,Plazo,Fecha_Reporte,Zonas,Ejecutivo
0,Grupo Bimbo,4.117590e+06,MXN,100000,2019-08-27,2023-04-24,1336,2025-09-29,Norte,Luis Manuel Ana María Vera Domínguez
1,Walmart De México,1.285973e+08,USD,100001,2017-04-10,2022-02-18,1775,2025-09-29,Sur,Eloisa Salinas Aguilar
2,Liverpool,2.134391e+08,EUR,100002,2017-04-21,1900-01-01,0,2025-09-29,Norte,Dr. Virginia Pelayo
3,Grupo Lala,2.372735e+07,USD,100003,2023-11-24,1900-01-01,0,2025-09-29,Sur,Juana Antonia Lira Samaniego
4,Femsa,1.534085e+08,EUR,100004,2024-10-30,1900-01-01,0,2025-09-29,Centro,Gabino Mariano Laureano Puente
...,...,...,...,...,...,...,...,...,...,...
9995,Bbva México,7.649614e+06,MXN,109995,2021-01-02,2023-03-01,788,2025-09-29,Centro,Víctor Durán de la Crúz
9996,Soriana,1.578366e+06,MXN,109996,2017-02-28,2022-02-12,1810,2025-09-29,Norte,Alfonso Natalia Sevilla
9997,Walmart De México,2.057032e+06,MXN,109997,2020-05-19,1900-01-01,0,2025-09-29,Sur,Eloisa Salinas Aguilar
9998,Grupo Bimbo,6.555259e+06,MXN,109998,2017-05-13,2019-12-24,955,2025-09-29,Norte,Luis Manuel Ana María Vera Domínguez


In [45]:
hoy = pd.Timestamp.today()
df_activos=df_bank_final[df_bank_final['Fecha_Fin'] > hoy].groupby("Nombre_Empresa").size()
df_activos



,0
Nombre_Empresa,
Alsea,2
Cemex,3
Grupo México,2
Soriana,1
Telmex,1


In [52]:
total=df_bank_final["Monto_Credito"].sum()
total_por_divisa=df_bank_final[["Monto_Credito","Divisa"]].groupby("Divisa")["Monto_Credito"].sum()
porcentaje=(total_por_divisa/total)*100
print(porcentaje)



Divisa
EUR    52.803691
MXN     2.357795
USD    44.838514
Name: Monto_Credito, dtype: float64


In [55]:
#Agregar una columna con el monto total de deuda de cada empresa
df_monto_total_empresa=df_bank_final[["Nombre_Empresa","Monto_Credito"]].groupby("Nombre_Empresa")["Monto_Credito"].sum().reset_index()
df_monto_total_empresa

,Nombre_Empresa,Monto_Credito
0,Alsea,4.138519e+10
1,América Móvil,4.637805e+10
2,Banorte,4.832633e+10
3,Bbva México,4.916227e+10
4,Cemex,4.675240e+10
5,Femsa,5.119268e+10
6,Grupo Bimbo,4.469775e+10
7,Grupo Carso,4.651391e+10
8,Grupo Lala,4.669941e+10
9,Grupo México,4.611247e+10


In [60]:
df_ordenado=df_bank_final.sort_values("Fecha_Inicio", ascending=False).groupby("Nombre_Empresa").last()
df_ordenado

,Monto_Credito,Divisa,Operacion,Fecha_Inicio,Fecha_Fin,Plazo,Fecha_Reporte,Zonas,Ejecutivo
Nombre_Empresa,,,,,,,,,
Alsea,9.884533e+06,MXN,108009,2015-09-09,2021-12-26,2300,2025-09-29,Sur,Sandra Rodolfo Amador
América Móvil,1.696858e+06,MXN,100582,2015-08-28,2020-09-20,1850,2025-09-29,Centro,Cristobal Miguel Ángel Armenta Mayorga
Banorte,1.144608e+08,EUR,108073,2015-08-31,1900-01-01,0,2025-09-29,Centro,Eloisa Salinas Aguilar
Bbva México,7.261334e+06,MXN,105315,2015-01-09,2025-01-06,3650,2025-09-29,Sur,Víctor Durán de la Crúz
Cemex,1.551235e+08,USD,107441,2015-04-11,2021-11-18,2413,2025-09-29,Norte,Francisco Bustos Nájera
Femsa,2.041206e+07,USD,106682,2015-07-12,2019-10-02,1543,2025-09-29,Norte,Gabino Mariano Laureano Puente
Grupo Bimbo,8.773058e+06,MXN,101235,2015-08-30,1900-01-01,0,2025-09-29,Norte,Luis Manuel Ana María Vera Domínguez
Grupo Carso,7.384692e+07,USD,105419,2015-03-11,2020-03-31,1847,2025-09-29,Centro,Gloria Otero Sáenz
Grupo Lala,4.973692e+07,EUR,100071,2015-01-09,2022-03-01,2608,2025-09-29,Centro,Juana Antonia Lira Samaniego
